In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast 

In [36]:
labelling_file='labels_my-project-name_2024-03-30-01-29-11.csv'
df=pd.read_csv(labelling_file)
df

,26K5RJ3ZW2IIMR2KJFDXSDSKTHUW4E7NMINPCS2MHC6HAQ23BTJA.jpg,[None]
0,2ADGDWDYAXB4AK47FFOPLEDWXFIY7XMMRADYJAJAJRCCGP...,[Tower Bridge]
1,2AFWOO37IJZG4GOU5GMTBKNJX2OQTRQCOZB6YRD5Z72EJP...,[None]
2,2AUW6KOSKR24Q624P4BAXVOE3DDAZVO75A6UHZUBE4MREG...,[Big Ben]
3,2BD5NPKTUNCUIKKYTNJLYABRHG2AMG4GSQO67SM2FK5CM3...,[Tower Bridge]
4,2CMDM4PL4E6UW2YYX3THTWYO6ZLB73CHNMTWJCTJKOAU76...,[None]
...,...,...
898,ZUGAWZ7VC6DURHHYMIPAEU6WQUKRQNCSKL23RSZTI3NMXG...,[None]
899,ZWFZWTA6UQV6BPHDXUWANP4D2ROCURCU564QN2ADH27KCB...,[Tower Of London]
900,ZWKU32WWE2IPEEWZGGYYCSFLU6PNGZIF52UYE6ROLGE673...,"[Lastminute.com London Eye,Palace Of Westminst..."
901,ZXNYLU5WQHHLKBJ6IZ6XF4VPRXWB4CBU53PSXGSIOX57VG...,"[Lastminute.com London Eye,Big Ben]"


In [37]:
#Rename columns to match the format of the data + Remove file extension from the image_id
df.columns = ['image_id', 'ground_truth']
df['image_id'] = df['image_id'].str.split('.').str[0]
df

,image_id,ground_truth
0,2ADGDWDYAXB4AK47FFOPLEDWXFIY7XMMRADYJAJAJRCCGP...,[Tower Bridge]
1,2AFWOO37IJZG4GOU5GMTBKNJX2OQTRQCOZB6YRD5Z72EJP...,[None]
2,2AUW6KOSKR24Q624P4BAXVOE3DDAZVO75A6UHZUBE4MREG...,[Big Ben]
3,2BD5NPKTUNCUIKKYTNJLYABRHG2AMG4GSQO67SM2FK5CM3...,[Tower Bridge]
4,2CMDM4PL4E6UW2YYX3THTWYO6ZLB73CHNMTWJCTJKOAU76...,[None]
...,...,...
898,ZUGAWZ7VC6DURHHYMIPAEU6WQUKRQNCSKL23RSZTI3NMXG...,[None]
899,ZWFZWTA6UQV6BPHDXUWANP4D2ROCURCU564QN2ADH27KCB...,[Tower Of London]
900,ZWKU32WWE2IPEEWZGGYYCSFLU6PNGZIF52UYE6ROLGE673...,"[Lastminute.com London Eye,Palace Of Westminst..."
901,ZXNYLU5WQHHLKBJ6IZ6XF4VPRXWB4CBU53PSXGSIOX57VG...,"[Lastminute.com London Eye,Big Ben]"


In [38]:
data_evaluation='edited_detection_results_vision_openai.csv'
df_1=pd.read_csv(data_evaluation) 

df_1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   accommodation_id          916 non-null    int64  
 1   longitude                 916 non-null    float64
 2   latitude                  916 non-null    float64
 3   driving_duration          916 non-null    int64  
 4   walking_duration          916 non-null    int64  
 5   transit_duration          916 non-null    int64  
 6   image_id                  916 non-null    object 
 7   image                     916 non-null    object 
 8   tag_name                  916 non-null    object 
 9   probability               916 non-null    float64
 10  accommodation_name        916 non-null    object 
 11  dt                        916 non-null    object 
 12  distance_to_bigben        916 non-null    float64
 13  description               558 non-null    object 
 14  score     

In [39]:
df_1['detections_vision_openai'].unique()


array(['None', 'The Shard', 'Kensington Palace',
       '30 St Mary Axe (The Gherkin)', 'Error in API call', 'Big Ben',
       'Tower Bridge', 'The British Museum', 'Trafalgar Square',
       'Lastminute.com London Eye', 'Westminster Bridge', 'Big Ben.',
       'Palace Of Westminster', 'Westminster Abbey',
       'Palace of Westminster.', 'Palace of Westminster', 'Tower Bridge.',
       'Tower Of London', 'Tower of London', 'Wembley Stadium'],
      dtype=object)

In [69]:
df_1['description'].unique()

array(['None', 'Big Ben', 'Westminster Bridge', 'Tower Bridge',
       'Lastminute.com London Eye', 'Tower Of London', 'Trafalgar Square',
       'The British Museum', 'The Shard', '30 St Mary Axe (the Gherkin)',
       'Palace Of Westminster', 'Wembley Stadium'], dtype=object)

In [71]:
# Drop images where "Error in API call" and get % of overall dataset
# For Any row in detectionss_vision_openai, replace '.' at the end of the string with empty space and Capatilize the first letter of every substring
def standardize_format(s):
    if pd.isna(s):
        return 'None'
    if s == 'Westminster Abbey':
        return 'Palace Of Westminster'
    # Remove full stops at the end
    if s== 'Kensington Palace':
        return 'None'
    if s == '30 St Mary Axe (the Gherkin)':
        return '30 St Mary Axe (The Gherkin)'
    s = s.rstrip('.')
    # Capitalize the first letter of each substring
    s = ' '.join(word.capitalize() for word in s.split())
    return s

# Apply the function to the DataFrame column
df_1['detections_vision_openai'] = df_1['detections_vision_openai'].apply(standardize_format)
df_1['description'] = df_1['description'].apply(standardize_format)



In [72]:
df_1['detections_vision_openai'].unique()

array(['None', 'The Shard', '30 St Mary Axe (The Gherkin)', 'Big Ben',
       'Tower Bridge', 'The British Museum', 'Trafalgar Square',
       'Lastminute.com London Eye', 'Westminster Bridge',
       'Palace Of Westminster', 'Tower Of London', 'Wembley Stadium'],
      dtype=object)

merged_df = pd.merge(df_2, df_1, on='image_id', how='inner')
columns_to_drop = merged_df.columns[[2, 18]].tolist()
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df.rename(columns={'description': 'detections_google_vision'}, inplace=True)

merged_df

In [73]:
df_1['description'].unique()

array(['None', 'Big Ben', 'Westminster Bridge', 'Tower Bridge',
       'Lastminute.com London Eye', 'Tower Of London', 'Trafalgar Square',
       'The British Museum', 'The Shard', '30 St Mary Axe (The Gherkin)',
       'Palace Of Westminster', 'Wembley Stadium'], dtype=object)

In [75]:
count = df_1['detections_vision_openai'].str.contains('Error In Api Call').sum()
print(count)
df_1 = df_1[~df_1['detections_vision_openai'].str.contains('Error In Api Call')]

0


In [76]:
df_1['detections_vision_openai'].unique()

array(['None', 'The Shard', '30 St Mary Axe (The Gherkin)', 'Big Ben',
       'Tower Bridge', 'The British Museum', 'Trafalgar Square',
       'Lastminute.com London Eye', 'Westminster Bridge',
       'Palace Of Westminster', 'Tower Of London', 'Wembley Stadium'],
      dtype=object)

In [77]:
merged_df = pd.merge(df, df_1, on='image_id', how= 'right')
merged_df


,image_id,ground_truth,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai
0,DX64WCVN23LSWFWPBOWT5YGBMO45N3SF3HBQLCFHXQUJCV...,[None],1504047,-0.12759,51.52702,13,33,19,https://imgcy.trivago.com/partner-images/1d/fd...,building,0.9999,Studios 2 Let Serviced Apartments,2024-03-07,1.998302,None,0.750398,0.124018,3.288819,None
1,RB77EISJ5MQDUEGFZSGFH3EGQQNEDPHANSIYFNAFJJKIN6...,[None],10405908,-0.12750,51.50722,4,14,14,https://imgcy.trivago.com/partner-images/88/7f...,building,1.0000,Lovely Family Apartments For Upto 6 Near Oxfor...,2024-03-07,0.820251,None,0.413374,0.468961,4.299603,None
2,SWPQ2FXDSA2PH4KDMR56KNBPFJZ5C77DBKGBBAA4BPG5CG...,[None],18426,-0.18294,51.49556,21,77,17,https://imgcy.trivago.com/partner-images/95/9f...,building,1.0000,Meliá London Kensington Meliá Collection,2024-03-07,4.866410,None,0.632710,0.451684,5.174545,None
3,3DL2RXNO4NVWZFBC4L2YATHITDKB2DDWBF5LQ6JLO2C6GM...,[None],22032936,-0.12750,51.50722,4,14,11,https://imgcy.trivago.com/partner-images/d8/d7...,building,1.0000,Victorian Conversion Garden Flat,2024-03-07,0.820251,None,0.579433,0.213040,2.592985,The Shard
4,4MLPB64JFHPSIM2YTAFY4U5HCCVUIUABRH7GUVMVTJZKKO...,[None],20648498,-0.12750,51.50722,4,14,9,https://imgcy.trivago.com/partner-images/e3/16...,building,1.0000,Stunning Central Richmond With Garden,2024-03-07,0.820251,None,0.650984,0.381180,4.568479,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,3ERUOOVFDLAGJLMBZHU22QRWRFUPS4ZQT36ZWD5XMIAHSX...,[Palace Of Westminster],46832,-0.10219,51.50610,9,23,16,https://imgcy.trivago.com/partner-images/d9/23...,building,0.9999,Holiday Inn Express London - Southwark,2024-03-07,1.114754,Palace Of Westminster,0.376706,0.361036,4.262479,Palace Of Westminster
907,HHWELDHJZU6GDBQT64L4BQMRMQFN24R4VHKAV4NLWPXD6T...,"[Big Ben,Palace Of Westminster]",46539,-0.07737,51.51093,9,40,14,https://imgcy.trivago.com/partner-images/39/ec...,building,0.9999,Bridgestreet Pepys Street,2024-03-07,2.727606,Palace Of Westminster,0.465586,0.290676,3.912997,Big Ben
908,YOCE5VDTKT3X6YCMRVWEMD36FJL2MK3AVYRMUZSBB46FFL...,"[Palace Of Westminster,Big Ben]",46461,-0.13245,51.50752,10,19,11,https://imgcy.trivago.com/partner-images/c3/84...,building,0.9972,Sofitel London St James,2024-03-07,1.129889,Palace Of Westminster,0.708477,0.402796,4.163838,Palace Of Westminster
909,GXVVATZDDOWNRD5CN7CUVAE7XGOJ3JNLY5H64EMT2RX7TA...,"[Big Ben,Westminster Bridge]",8003044,-0.09626,51.51041,5,21,14,https://imgcy.trivago.com/partner-images/35/eb...,building,0.9999,Superior 2 Bedroom 2 Bath Apartment St Pauls L...,2024-03-07,1.419610,Palace Of Westminster,0.671675,0.462791,6.533917,Big Ben


In [78]:
null_rows = merged_df[merged_df['ground_truth'].isna()]
null_rows


,image_id,ground_truth,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai
110,NF3QO6RXAONXIJQY3K43IFDAYGY6W5QDBM3EBAXOJL6RCC...,NaN,23573472,-0.12750,51.50722,4,14,10,https://imgcy.trivago.com/partner-images/69/77...,building,0.9769,"Compact Bayswater 1br W/ Modern Fit Out, Nr. N...",2024-03-07,0.820251,None,0.232687,0.308111,6.750617,None
161,26K5RJ3ZW2IIMR2KJFDXSDSKTHUW4E7NMINPCS2MHC6HAQ...,NaN,25279996,-0.12750,51.50722,4,14,10,https://imgcy.trivago.com/partner-images/d7/95...,kids-area,0.9989,Pass The Keys Apartment With A Private Garden ...,2024-03-07,0.820251,None,0.453807,0.519399,7.264880,None
170,4KN6DU776WPWR4CKZA42V6I7GZZU7UBK62HN3CRO7OZVAP...,NaN,12701,-0.12573,51.52014,10,22,16,https://imgcy.trivago.com/partner-images/e2/9b...,building,0.9824,Hotel The Montague on the Gardens,2024-03-07,1.252002,None,0.310829,0.479584,4.059471,None
213,KLN6PX457DB5JNRS3QT4QRK5YOKJVSX7IZRA4FG5ESZCHG...,NaN,11050412,-0.14183,51.50878,14,29,18,https://imgcy.trivago.com/partner-images/52/db...,building,0.9998,Stunning Mayfair £8 Million 3 Bedroom 3.5 Bath...,2024-03-07,1.742939,None,0.476185,0.353826,4.000016,Trafalgar Square
343,4B6MXYFH36W4CAE4YPUSPXUG4EMNYWQZZO37FW7DNGJTNA...,NaN,16816328,-0.12750,51.50722,4,14,10,https://imgcy.trivago.com/partner-images/e0/7c...,building,0.9999,Chelsea Harbour Flat,2024-03-07,0.820251,None,0.429535,0.217828,2.966757,None
441,L675IHLX4BEA43G7EXYRWJYY6GTXL5STR2G37VQDKY64US...,NaN,12641,-0.12510,51.52407,11,27,16,https://imgcy.trivago.com/partner-images/5f/bf...,building,0.9972,Holiday Inn London - Bloomsbury,2024-03-07,1.630048,Big Ben,0.775605,0.457238,5.191299,Westminster Bridge
512,KL2ESWURIUHRJ2DQX62HQDKTKD3WQASUH57NJZZ6US23RJ...,NaN,2498036,-0.08686,51.50411,9,38,23,https://imgcy.trivago.com/partner-images/52/f4...,building,0.9998,"Shangri-La Hotel, At The Shard, London",2024-03-07,2.183695,Tower Bridge,0.841503,0.200339,7.662652,Tower Bridge
570,YLF36RZQH6GPYK2IW325FYLHFODTQAVPIOKQYSM32AQUFU...,NaN,30600602,-0.12750,51.50722,4,14,10,https://imgcy.trivago.com/partner-images/c2/cb...,building,0.9999,Bright And Lovely 1 Bedroom - 5 Min From Tower...,2024-03-07,0.820251,Tower Bridge,0.575662,0.339165,5.483009,Tower Bridge
626,L5JMPA42AL5XLA7TS6DBY2VN24AJFFDWPG23EPN2WPHWYS...,NaN,1535559,-0.12502,51.50613,3,11,11,https://imgcy.trivago.com/hotelier-images/5f/5...,building,0.9988,Corinthia Hotel London,2024-03-07,0.739382,Lastminute.com London Eye,0.218882,0.972604,9.786182,None
646,NEYNJ4W4N3YGWYGT65INIFE23EIKNXDRWTF4S6QJ4J3JDI...,NaN,10823816,-0.11345,51.50029,11,21,13,https://imgcy.trivago.com/partner-images/69/30...,beach,0.9325,Best Deal In Central London,2024-03-07,1.142784,Lastminute.com London Eye,0.648279,0.240010,7.014555,Big Ben


In [79]:
updates = {
    110: '[None]',
    161: '[None]',
    170: '[None]',
    213: '[None]',
    343: '[None]',
    441: '[Westminster Bridge,Big Ben,Palace Of Westminster]',
    512: '[The Shard,Tower Bridge]',
    570: '[Tower Bridge]',
    626: '[None]',
    646: '[Big Ben, Lastminute.com London Eye]',
    740: '[Tower Of London]',
    771: '[None]',
    893: '[The Shard]'
}




In [80]:
for index, value in updates.items():
    merged_df.at[index, 'ground_truth'] = value

merged_df.loc[[443, 514, 572, 648, 742, 898]]  # Display the updated rows to verify the changes

,image_id,ground_truth,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai
443,SKSGDI523KNGX52RRA6SER3356L72WEHA7PYMZ6SPSQ24N...,"[Westminster Bridge,Palace Of Westminster,Big ...",30440764,-0.12750,51.50722,4,14,10,https://imgcy.trivago.com/partner-images/92/a4...,building,0.9999,Pass The Keys | Cheerful And Cosy Apartment In...,2024-03-07,0.820251,Big Ben,0.486329,0.420697,4.660265,Big Ben
514,SLJLDRFVMHRHWMSJR5FIH5P6QNUWVSKVZ4ZPESU6AOI3SD...,"[Tower Bridge,The Shard]",2498036,-0.08686,51.50411,9,38,23,https://imgcy.trivago.com/partner-images/92/d2...,building,0.9998,"Shangri-La Hotel, At The Shard, London",2024-03-07,2.183695,Tower Bridge,0.841503,0.222265,3.399859,Tower Bridge
572,PC2Q3BIXJXAKS67M47HODAMRKNSGUNTN3GRV4HLA7QGJMX...,"[30 St Mary Axe (The Gherkin),None]",18172382,-0.11265,51.51284,4,7,7,https://imgcy.trivago.com/partner-images/78/b5...,building,0.9999,Eldon Chambers Flat 1,2024-03-07,0.397038,Tower Bridge,0.587840,0.190857,2.660432,Tower Of London
648,GTA6S6SQI2ZLK32TOM2PAQCOEVF7LRFSILP3NUJW4467EI...,[Lastminute.com London Eye],31540182,-0.07798,51.50149,12,48,17,https://imgcy.trivago.com/partner-images/34/c1...,building,0.9999,Tower Bridge Modern 2bed 2bath Apartment By Lo...,2024-03-07,2.860236,Lastminute.com London Eye,0.511336,0.151479,2.856157,Lastminute.com London Eye
742,SCYLZKA62G6RDXV52HJZDHGV263JQID7AUUAXKARY3U2RT...,[Tower Of London],30418188,-0.07358,51.51710,15,48,17,https://imgcy.trivago.com/hotelier-images/90/b...,building,0.9902,Bob W Commercial House,2024-03-07,3.081553,Tower Of London,0.473048,0.100976,1.030194,Tower Of London
898,I5SSLN4TSF5EJOJXEN62PBOVYC6FTVNOTPN22E35JJVGFX...,[30 St Mary Axe (The Gherkin)],18153176,-0.08264,51.51498,10,35,17,https://imgcy.trivago.com/partner-images/47/65...,building,0.9999,"Apartment 4, 48 Bishopsgate",2024-03-07,2.417339,30 St Mary Axe (The Gherkin),0.290306,0.882653,9.223202,30 St Mary Axe (The Gherkin)


In [81]:
null_rows = merged_df[merged_df['ground_truth'].isna()]
null_rows

,image_id,ground_truth,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai


In [82]:
def extract_first_class(s):
    # Check if there's a comma, indicating multiple classes
    if ',' in s:
        # Extract the first class: strip the brackets, split by comma, and take the first element
        first_class = s.strip('[]').split(',')[0].strip()
    else:
        # If there's no comma, it's a single class. Remove brackets and return the class directly.
        first_class = s.strip('[]').strip()
    return first_class

# Apply the function to the 'ground_truth' column
merged_df['ground_truth_single'] = merged_df['ground_truth'].apply(extract_first_class)

merged_df

,image_id,ground_truth,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time,response_time_vision,detections_vision_openai,ground_truth_single
0,DX64WCVN23LSWFWPBOWT5YGBMO45N3SF3HBQLCFHXQUJCV...,[None],1504047,-0.12759,51.52702,13,33,19,https://imgcy.trivago.com/partner-images/1d/fd...,building,0.9999,Studios 2 Let Serviced Apartments,2024-03-07,1.998302,None,0.750398,0.124018,3.288819,None,None
1,RB77EISJ5MQDUEGFZSGFH3EGQQNEDPHANSIYFNAFJJKIN6...,[None],10405908,-0.12750,51.50722,4,14,14,https://imgcy.trivago.com/partner-images/88/7f...,building,1.0000,Lovely Family Apartments For Upto 6 Near Oxfor...,2024-03-07,0.820251,None,0.413374,0.468961,4.299603,None,None
2,SWPQ2FXDSA2PH4KDMR56KNBPFJZ5C77DBKGBBAA4BPG5CG...,[None],18426,-0.18294,51.49556,21,77,17,https://imgcy.trivago.com/partner-images/95/9f...,building,1.0000,Meliá London Kensington Meliá Collection,2024-03-07,4.866410,None,0.632710,0.451684,5.174545,None,None
3,3DL2RXNO4NVWZFBC4L2YATHITDKB2DDWBF5LQ6JLO2C6GM...,[None],22032936,-0.12750,51.50722,4,14,11,https://imgcy.trivago.com/partner-images/d8/d7...,building,1.0000,Victorian Conversion Garden Flat,2024-03-07,0.820251,None,0.579433,0.213040,2.592985,The Shard,None
4,4MLPB64JFHPSIM2YTAFY4U5HCCVUIUABRH7GUVMVTJZKKO...,[None],20648498,-0.12750,51.50722,4,14,9,https://imgcy.trivago.com/partner-images/e3/16...,building,1.0000,Stunning Central Richmond With Garden,2024-03-07,0.820251,None,0.650984,0.381180,4.568479,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,3ERUOOVFDLAGJLMBZHU22QRWRFUPS4ZQT36ZWD5XMIAHSX...,[Palace Of Westminster],46832,-0.10219,51.50610,9,23,16,https://imgcy.trivago.com/partner-images/d9/23...,building,0.9999,Holiday Inn Express London - Southwark,2024-03-07,1.114754,Palace Of Westminster,0.376706,0.361036,4.262479,Palace Of Westminster,Palace Of Westminster
907,HHWELDHJZU6GDBQT64L4BQMRMQFN24R4VHKAV4NLWPXD6T...,"[Big Ben,Palace Of Westminster]",46539,-0.07737,51.51093,9,40,14,https://imgcy.trivago.com/partner-images/39/ec...,building,0.9999,Bridgestreet Pepys Street,2024-03-07,2.727606,Palace Of Westminster,0.465586,0.290676,3.912997,Big Ben,Big Ben
908,YOCE5VDTKT3X6YCMRVWEMD36FJL2MK3AVYRMUZSBB46FFL...,"[Palace Of Westminster,Big Ben]",46461,-0.13245,51.50752,10,19,11,https://imgcy.trivago.com/partner-images/c3/84...,building,0.9972,Sofitel London St James,2024-03-07,1.129889,Palace Of Westminster,0.708477,0.402796,4.163838,Palace Of Westminster,Palace Of Westminster
909,GXVVATZDDOWNRD5CN7CUVAE7XGOJ3JNLY5H64EMT2RX7TA...,"[Big Ben,Westminster Bridge]",8003044,-0.09626,51.51041,5,21,14,https://imgcy.trivago.com/partner-images/35/eb...,building,0.9999,Superior 2 Bedroom 2 Bath Apartment St Pauls L...,2024-03-07,1.419610,Palace Of Westminster,0.671675,0.462791,6.533917,Big Ben,Big Ben


In [83]:
# Replace brackets at the beginning and end of each string, but keep the content
merged_df['ground_truth_multiple'] = merged_df['ground_truth'].str.replace(r'^\[|\]$', '', regex=True)

merged_df['ground_truth_multiple']

0                               None
1                               None
2                               None
3                               None
4                               None
                   ...              
906            Palace Of Westminster
907    Big Ben,Palace Of Westminster
908    Palace Of Westminster,Big Ben
909       Big Ben,Westminster Bridge
910                  Wembley Stadium
Name: ground_truth_multiple, Length: 911, dtype: object

In [84]:
merged_df['description'] = merged_df['description'].astype(str)
merged_df['detections_vision_openai'] = merged_df['detections_vision_openai'].astype(str)

In [85]:
merged_df.to_csv('Google+Openai_Evaluation_Input.csv')